<a href="https://colab.research.google.com/github/Segn11/zindi-financial-inclusion/blob/prediction3/financial_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv('/content/Train (6).csv')
test = pd.read_csv('/content/Test (5).csv')

In [52]:
df.head()


,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [70]:

df_id = df['uniqueid'].copy()
test_id = test["uniqueid"].copy()

# Prepare text data
X_df_raw = df.drop(columns=["uniqueid", "bank_account"]).copy()
X_te_raw = test.drop(columns=["uniqueid"]).copy()

# Combine for vectorization
all_data = pd.concat([X_df_raw, X_te_raw], axis=0).reset_index(drop=True)


In [57]:
all_data.isna().sum()

,0
country,0
year,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0
marital_status,0
education_level,0


In [58]:
all_data.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [71]:
cat_cols = all_data.select_dtypes(include="object").columns
cat_cols = list(cat_cols)
print(cat_cols)

['country', 'location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head', 'marital_status', 'education_level', 'job_type']


In [72]:
num_cols = all_data.select_dtypes(include="number").columns
num_cols = list(num_cols)
print(num_cols)

['year', 'household_size', 'age_of_respondent']


In [73]:
#feature engineering
edu_map = {
    "No formal education": 0,
    "Primary education": 1,
    "Secondary education": 2,
    "Tertiary education": 3
}
all_data["education_num"] = all_data["education_level"].map(edu_map)


In [74]:
#feature engineering
all_data["is_large_family"] = (all_data["household_size"] > 5).astype(int)
all_data["household_per_age"] = all_data["household_size"] / (all_data["age_of_respondent"] + 1)
all_data["rural_and_low_education"] = ((all_data["location_type"] == "Rural") & (all_data["education_level"].isin(["No formal education", "Primary education"]))).astype(int)



In [75]:
# Age weighted by household size
all_data["age_times_household"] = all_data["age_of_respondent"] * all_data["household_size"]

# Age and education interaction
all_data["age_times_education"] = all_data["age_of_respondent"] * all_data["education_num"]

# Large family in rural area
all_data["rural_large_family"] = all_data["is_large_family"] * (all_data["location_type"] == "Rural").astype(int)

# Age of respondent * rural & low education
all_data["age_rural_low_edu"] = all_data["age_of_respondent"] * all_data["rural_and_low_education"]
# Household size per education level (already exists as household_per_age)
# You could also do:
all_data["household_per_education"] = all_data["household_size"] / (all_data["education_num"] + 1)

# Age per household member
all_data["age_per_person"] = all_data["age_of_respondent"] / (all_data["household_size"] + 1)
# Is adult (age > 25)
all_data["is_adult"] = (all_data["age_of_respondent"] > 25).astype(int)

# Is middle-aged (25-50)
all_data["is_middle_aged"] = ((all_data["age_of_respondent"] > 25) & (all_data["age_of_respondent"] <= 50)).astype(int)

# Rural AND no cellphone
all_data["rural_no_cellphone"] = ((all_data["location_type"] == "Rural") & (all_data["cellphone_access"] == "No")).astype(int)

# Married AND large family
all_data["married_large_family"] = ((all_data["marital_status"] == "Married/Living together") & (all_data["is_large_family"]==1)).astype(int)


In [ ]:
all_data["education_job"].nunique()


59

In [63]:
edu_map = {
    "No formal education": 0,
    "Primary education": 1,
    "Secondary education": 2,
    "Tertiary education": 3
}
all_data["education_num"] = all_data["education_level"].map(edu_map)


In [76]:
all_data.isna().sum()

,0
country,0
year,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0
marital_status,0
education_level,0


In [77]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   year                     33610 non-null  int64  
 2   location_type            33610 non-null  object 
 3   cellphone_access         33610 non-null  object 
 4   household_size           33610 non-null  int64  
 5   age_of_respondent        33610 non-null  int64  
 6   gender_of_respondent     33610 non-null  object 
 7   relationship_with_head   33610 non-null  object 
 8   marital_status           33610 non-null  object 
 9   education_level          33610 non-null  object 
 10  job_type                 33610 non-null  object 
 11  education_num            32395 non-null  float64
 12  is_large_family          33610 non-null  int64  
 13  household_per_age        33610 non-null  float64
 14  rural_and_low_educatio

In [78]:
#handling missing values
all_data["education_num"].fillna(0, inplace=True)
all_data["age_times_education"].fillna(0, inplace=True)
all_data["household_per_education"].fillna(0, inplace=True)



/tmp/ipython-input-1965310011.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data["education_num"].fillna(0, inplace=True)
/tmp/ipython-input-1965310011.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [79]:
#feature selection
all_data.drop(columns=["year"], inplace=True)


In [86]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   location_type            33610 non-null  object 
 2   cellphone_access         33610 non-null  object 
 3   household_size           33610 non-null  float64
 4   age_of_respondent        33610 non-null  float64
 5   gender_of_respondent     33610 non-null  object 
 6   relationship_with_head   33610 non-null  object 
 7   marital_status           33610 non-null  object 
 8   education_level          33610 non-null  object 
 9   job_type                 33610 non-null  object 
 10  education_num            33610 non-null  float64
 11  is_large_family          33610 non-null  float64
 12  household_per_age        33610 non-null  float64
 13  rural_and_low_education  33610 non-null  int64  
 14  age_times_household   

In [82]:
num_cols = all_data.select_dtypes(include=[np.number]).columns

# Compute correlation matrix
corr_matrix = all_data[num_cols].corr().abs()  # absolute value

# Find highly correlated pairs
high_corr = np.where(corr_matrix > 0.9)
high_corr_pairs = [(num_cols[i], num_cols[j], corr_matrix.iloc[i, j])
                   for i, j in zip(*high_corr) if i < j]

print("Highly correlated feature pairs (corr>0.9):")
for f1, f2, corr_val in high_corr_pairs:
    print(f"{f1} - {f2}: {corr_val:.2f}")

Highly correlated feature pairs (corr>0.9):


In [83]:
num_cols = all_data.select_dtypes(include="number").columns
skewed = all_data[num_cols].apply(lambda x: x.dropna().skew()).abs()
skew_cols = skewed[skewed > 0.75].index
all_data[skew_cols] = np.log1p(all_data[skew_cols])

In [84]:
all_data.head()

,country,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,...,age_times_household,age_times_education,rural_large_family,age_rural_low_edu,household_per_education,age_per_person,is_adult,is_middle_aged,rural_no_cellphone,married_large_family
0,Kenya,Rural,Yes,1.386294,3.218876,Female,Spouse,Married/Living together,Secondary education,Self employed,...,4.290459,3.891820,0.0,0.000000,0.693147,1.945910,0.000000,0,0.000000,0.0
1,Kenya,Rural,No,1.791759,4.262680,Female,Head of Household,Widowed,No formal education,Government Dependent,...,5.860786,0.000000,0.0,4.262680,1.791759,2.538974,0.693147,0,0.693147,0.0
2,Kenya,Urban,Yes,1.791759,3.295837,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,...,4.875197,0.000000,0.0,0.000000,0.000000,1.673976,0.693147,1,0.000000,0.0
3,Kenya,Rural,Yes,1.791759,3.555348,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,...,5.141664,3.555348,0.0,3.555348,1.252763,1.897120,0.693147,1,0.000000,0.0
4,Kenya,Urban,No,2.197225,3.295837,Male,Child,Single/Never Married,Primary education,Informally employed,...,5.342334,3.295837,0.0,0.000000,1.609438,1.358123,0.693147,1,0.000000,0.0


In [85]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   location_type            33610 non-null  object 
 2   cellphone_access         33610 non-null  object 
 3   household_size           33610 non-null  float64
 4   age_of_respondent        33610 non-null  float64
 5   gender_of_respondent     33610 non-null  object 
 6   relationship_with_head   33610 non-null  object 
 7   marital_status           33610 non-null  object 
 8   education_level          33610 non-null  object 
 9   job_type                 33610 non-null  object 
 10  education_num            33610 non-null  float64
 11  is_large_family          33610 non-null  float64
 12  household_per_age        33610 non-null  float64
 13  rural_and_low_education  33610 non-null  int64  
 14  age_times_household   

In [87]:
from sklearn.preprocessing import LabelEncoder
#float_array = all_data[["household_size", "age_of_respondent", "year"]].values.astype(float)

# List of categorical columns for One-Hot Encoding
cat_cols = [
    "relationship_with_head",
    "marital_status",
    "education_level",
    "job_type",
    "country",

]

# Keep only columns that exist in all_data
cat_cols = [col for col in cat_cols if col in all_data.columns]

# One-Hot Encoding
all_data = pd.get_dummies(all_data, columns=cat_cols, prefix_sep="_")

# Label encoding for small/binary categorical columns
le = LabelEncoder()
for col in ["location_type", "cellphone_access", "gender_of_respondent"]:
    if col in all_data.columns:
        all_data[col] = le.fit_transform(all_data[col].astype(str))


In [88]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 50 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   location_type                                    33610 non-null  int64  
 1   cellphone_access                                 33610 non-null  int64  
 2   household_size                                   33610 non-null  float64
 3   age_of_respondent                                33610 non-null  float64
 4   gender_of_respondent                             33610 non-null  int64  
 5   education_num                                    33610 non-null  float64
 6   is_large_family                                  33610 non-null  float64
 7   household_per_age                                33610 non-null  float64
 8   rural_and_low_education                          33610 non-null  int64  
 9   age_times_household         

In [89]:
# If train['bank_account'] contains 'Yes'/'No'
y = df['bank_account'].map({'No': 0, 'Yes': 1})


In [90]:
n_train = X_df_raw.shape[0]
X_train = all_data.iloc[:n_train, :].copy()
X_test  = all_data.iloc[n_train:, :].copy()

In [91]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42
)



In [92]:
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score

model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_tr, y_tr)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2670, number of negative: 16149
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1200
[LightGBM] [Info] Number of data points in the train set: 18819, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.141878 -> initscore=-1.799780
[LightGBM] [Info] Start training from score -1.799780
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=7,
               n_estimators=500, random_state=42, subsample=0.8)

In [93]:
!pip install optuna

In [94]:
!pip install catboost

In [95]:
# ---------------------------------------------
# 0️⃣ Imports
# ---------------------------------------------
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# ---------------------------------------------
# 1️⃣ Split Data (if not already done)
# ---------------------------------------------
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------------------------------------
# 2️⃣ Define Optuna Objective Function
# ---------------------------------------------
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 2.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "eval_metric": "AUC",
        "verbose": 0,
        "auto_class_weights": "Balanced",  # handles imbalance in target
    }

    model = CatBoostClassifier(**params)
    model.fit(
        X_tr,
        y_tr,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        use_best_model=True
    )

    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    return auc

# ---------------------------------------------
# 3️⃣ Run Optuna Study
# ---------------------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Increase trials for deeper tuning

# ---------------------------------------------
# 4️⃣ Train Final Model with Best Parameters
# ---------------------------------------------
best_params = study.best_params
print("✅ Best Parameters:", best_params)

model = CatBoostClassifier(**best_params)
model.fit(
    X_tr,
    y_tr,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50,
    use_best_model=True,
)

# ---------------------------------------------
# 5️⃣ Evaluate on Validation Data
# ---------------------------------------------
preds = model.predict_proba(X_val)[:, 1]
val_mae = mean_absolute_error(y_val, preds)
auc = roc_auc_score(y_val, preds)

print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation ROC AUC: {auc:.4f}")


[I 2025-10-20 14:49:24,324] A new study created in memory with name: no-name-e5f9e29b-558f-4a0b-9271-121998ba129c
[I 2025-10-20 14:49:27,461] Trial 0 finished with value: 0.8606216182084884 and parameters: {'iterations': 793, 'learning_rate': 0.28900942268149626, 'depth': 9, 'l2_leaf_reg': 0.10130394495784915, 'bagging_temperature': 0.6040341628222913, 'random_strength': 0.11071877060968793, 'border_count': 71}. Best is trial 0 with value: 0.8606216182084884.
[I 2025-10-20 14:49:30,980] Trial 1 finished with value: 0.8730945956346914 and parameters: {'iterations': 439, 'learning_rate': 0.054814862627615056, 'depth': 5, 'l2_leaf_reg': 0.0011597048462651234, 'bagging_temperature': 0.6347878609147142, 'random_strength': 0.06014130015193242, 'border_count': 37}. Best is trial 1 with value: 0.8730945956346914.
[I 2025-10-20 14:49:33,785] Trial 2 finished with value: 0.8728963865036956 and parameters: {'iterations': 931, 'learning_rate': 0.26830027426892195, 'depth': 5, 'l2_leaf_reg': 0.4040

✅ Best Parameters: {'iterations': 864, 'learning_rate': 0.03555972970920252, 'depth': 5, 'l2_leaf_reg': 9.277256962089465, 'bagging_temperature': 0.29134628285047004, 'random_strength': 1.5942158280476346, 'border_count': 229}
0:	learn: 0.6582367	test: 0.6584006	best: 0.6584006 (0)	total: 7.14ms	remaining: 6.16s
1:	learn: 0.6326697	test: 0.6328304	best: 0.6328304 (1)	total: 14ms	remaining: 6.04s
2:	learn: 0.6088590	test: 0.6088099	best: 0.6088099 (2)	total: 20.5ms	remaining: 5.88s
3:	learn: 0.5869028	test: 0.5866053	best: 0.5866053 (3)	total: 27.2ms	remaining: 5.84s
4:	learn: 0.5657794	test: 0.5654608	best: 0.5654608 (4)	total: 33.9ms	remaining: 5.83s
5:	learn: 0.5442922	test: 0.5440742	best: 0.5440742 (5)	total: 40.7ms	remaining: 5.82s
6:	learn: 0.5267724	test: 0.5264585	best: 0.5264585 (6)	total: 47.9ms	remaining: 5.86s
7:	learn: 0.5104284	test: 0.5100459	best: 0.5100459 (7)	total: 54.3ms	remaining: 5.81s
8:	learn: 0.4924709	test: 0.4920747	best: 0.4920747 (8)	total: 61.1ms	remaining

In [ ]:
y_val_pred = model.predict(X_val)
y_val_prob = model.predict_proba(X_val)[:, 1]  # probability of class 1

# Since Zindi uses MAE
val_mae = mean_absolute_error(y_val, y_val_pred)
print("Validation MAE:", val_mae)

# Optional: ROC-AUC for additional insight
roc_auc = roc_auc_score(y_val, y_val_prob)
print("Validation ROC-AUC:", roc_auc)


Validation MAE: 0.09436769394261424
Validation ROC-AUC: 0.9136123903684933


In [96]:
model.fit(X_train, y)  # train on all training data
test_preds = model.predict(X_test)




0:	learn: 0.6567357	total: 7.75ms	remaining: 6.69s
1:	learn: 0.6305139	total: 15ms	remaining: 6.47s
2:	learn: 0.6061322	total: 21.9ms	remaining: 6.3s
3:	learn: 0.5840699	total: 29.3ms	remaining: 6.29s
4:	learn: 0.5620254	total: 36.3ms	remaining: 6.24s
5:	learn: 0.5423859	total: 43.3ms	remaining: 6.19s
6:	learn: 0.5245332	total: 53.8ms	remaining: 6.59s
7:	learn: 0.5100230	total: 60.7ms	remaining: 6.5s
8:	learn: 0.4913310	total: 70.9ms	remaining: 6.74s
9:	learn: 0.4760398	total: 78.4ms	remaining: 6.69s
10:	learn: 0.4622644	total: 85.6ms	remaining: 6.64s
11:	learn: 0.4517580	total: 93.6ms	remaining: 6.65s
12:	learn: 0.4414134	total: 101ms	remaining: 6.61s
13:	learn: 0.4316753	total: 110ms	remaining: 6.67s
14:	learn: 0.4225645	total: 118ms	remaining: 6.66s
15:	learn: 0.4147738	total: 125ms	remaining: 6.63s
16:	learn: 0.4049427	total: 133ms	remaining: 6.6s
17:	learn: 0.3978289	total: 140ms	remaining: 6.57s
18:	learn: 0.3916452	total: 147ms	remaining: 6.55s
19:	learn: 0.3852476	total: 155ms	

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({"uniqueid": test["uniqueid"] + " x " + test["country"],
                           "bank_account": test.bank_account})

In [97]:
# test_id: Series of unique IDs
# test_country: Series of country names corresponding to test IDs
test_country = test["country"].copy()  # make sure you have this column

# Prepare submission
submission = pd.DataFrame({
    "unique_id": test_id.astype(str) + " x " + test_country.astype(str),
    "bank_account": test_preds
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission saved!")


✅ Submission saved!


In [98]:
submission.head()

,unique_id,bank_account
0,uniqueid_6056 x Kenya,1
1,uniqueid_6060 x Kenya,1
2,uniqueid_6065 x Kenya,0
3,uniqueid_6072 x Kenya,0
4,uniqueid_6073 x Kenya,0


In [99]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>